# Importing Libraries

In [115]:
import tensorflow as tf
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Preparing the data | Fitting the Meta Learner

We have made rolling weekly predictions from the ARIMA and Prophet models to generate the training data for the meta learner that is a shallow Neural Network.

In [108]:

# 10th to 23rd December
# X_arima = [2207.36530701, 2257.15707498, 2307.79501891, 2245.13065557,
#        2372.27734087, 2373.40726702, 2355.35230453, 2367.60456701, 2238.30182499, 2215.10485251, 2274.34381155, 2330.84775477, 2232.67192813, 2240.01541005]
# X_prophet =  [2182.3626124180373, 2210.2838234090113, 2296.2371425582273, 2243.959025125425, 2297.190432186929, 2356.247945430332, 2327.082524589263, 2347.023993715967, 2278.958331906505, 2218.791015977442, 2251.6091463759954, 2302.0280756239026, 2245.6024807618214, 2213.2603075885277]

# 30th October to 1st January rolling forecast
X_arima = [1775.18625302, 1794.04090366, 1797.09998287, 1813.92964986,
       1789.70055924, 1785.98663876, 1805.74695235, 1820.25916716, 1826.31117716, 1857.28415192, 1810.45268919, 1842.8816632 , 1867.73512456, 1905.21646133, 1910.77751355, 1898.57016713, 1899.85633162, 2133.22390826, 2090.53557782, 2064.43079376, 2057.59643475, 2067.58090195, 1990.26410792, 2072.43338311, 1972.05757129, 1972.43239636, 1974.17307484, 2025.2760529, 2034.21426854, 1947.87376207, 2076.44823599, 2074.26203704, 2093.34556548, 2096.3372784 , 2075.78369081, 2039.3873142 , 2061.10074963, 2041.59587504, 2064.47672325, 2099.34782105, 2178.33348852, 2207.36698702, 2257.15917672, 2307.79982278, 2245.13348619, 2372.28316137, 2373.41312503, 2355.35868763, 2367.60672357, 2238.30140087, 2215.10412768, 2274.34282889, 2330.84637017, 2232.67123216, 2240.01438231, 2210.09439178, 2230.8902031 , 2190.68793382, 2215.03354658, 2253.06710504,
       2340.83766946, 2322.87380631, 2279.91918307]

X_prophet = [1714.5232071107423, 1737.8352781436458, 1787.404045472399, 1769.3385396584497, 1804.4727490237312, 1771.6139143389023, 1787.0853463753904, 1804.9278629278, 1832.442049876758, 1847.9213156569638, 1800.8372180141791, 1833.8551440923252, 1851.1908600578886, 1875.8766529499596, 1906.0693079754456, 1911.638887258888, 1894.6745554074466, 2016.6556893872882, 2090.9914966377905, 2067.6699689236293, 2059.1832500566634, 2022.120443439323, 2011.0169676555552, 2039.472168409271, 1973.3299415466222, 1973.0080701238353, 1985.5771835760088, 2005.836755317427, 2003.0764888709293, 1951.5195096481254, 1995.6264569431403, 2066.182741215848, 2044.5403700257393, 2095.006745360281, 2079.450107435048, 2054.506512313062, 2044.301877429153, 2031.8766369903624, 2055.370325584237, 2066.0231676042936, 2125.337055314533, 2181.333655659587, 2210.285514605764, 2296.218085845307, 2244.10433550828, 2297.119356910385, 2356.3056557671525, 2327.189806449233, 2348.1196645735413, 2278.9333921132948, 2218.787284161625, 2251.6621820402966, 2301.9852885090286, 2245.658965423589, 2213.283117809, 2199.6038405298714, 2250.278826682667, 2187.6232902138954, 2166.9237947120873, 2216.7738485777973, 2293.2861269535583, 2341.4798982451643, 2282.4565902614195]

X = np.array([X_arima, X_prophet]).T

# 10th to 23rd December
# Y = [2352.462646484375, 2224.578857421875, 2202.0380859375, 2260.648681640625, 2316.5791015625, 2219.33740234375, 2226.926513671875, 2196.481201171875, 2217.2734375, 2177.87255859375, 2201.911376953125, 2239.54248046875, 2326.52490234375, 2309.04638671875]

# 30th October to 1st January actual closing price
Y = [1810.088623046875, 1816.458984375, 1847.0897216796875, 1800.6209716796875, 1832.795166015625, 1857.6986083984375, 1894.15771484375, 1899.83740234375, 1888.124267578125, 1889.3223876953125, 2120.56103515625, 2078.289794921875, 2052.7138671875, 2045.18701171875, 2055.265380859375, 1979.0526123046875, 2060.408447265625, 1960.881591796875, 1961.28076171875, 1963.2850341796875, 2013.2044677734375, 2022.2391357421875, 1937.0667724609375, 2064.42529296875, 2062.210693359375, 2081.152099609375, 2084.4130859375, 2063.2861328125, 2027.4173583984375, 2049.338134765625, 2029.92919921875, 2052.55615234375, 2087.139892578125, 2165.7041015625, 2193.691650390625, 2243.2158203125, 2293.841796875, 2231.661376953125, 2357.57958984375, 2358.73193359375, 2341.17529296875, 2352.462646484375, 2224.578857421875, 2202.0380859375, 2260.648681640625, 2316.5791015625, 2219.33740234375, 2226.926513671875, 2196.481201171875, 2217.2734375, 2177.87255859375, 2201.911376953125, 2239.54248046875, 2326.52490234375, 2309.04638671875, 2265.692138671875,2272.561767578125,
 2231.46533203125,
 2378.739990234375,
 2347.566162109375,
 2300.690673828125,
 2292.0654296875,
 2281.47119140625  ]
# X as numpy array
X = np.array(np.log(X))
# Y as numpy array
Y = np.array(np.log(Y))

# Define the model
model = tf.keras.Sequential([
     tf.keras.layers.Dense(units=4, activation='relu', input_shape=(2,)),
     tf.keras.layers.Dense(units=1)
 ])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, Y,epochs=500)


Epoch 1/500
2/2 [==============================] - 5s 18ms/step - loss: 18.3845
Epoch 2/500
2/2 [==============================] - 0s 14ms/step - loss: 17.6946
Epoch 3/500
2/2 [==============================] - 0s 14ms/step - loss: 17.0166
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 16.3520
Epoch 5/500
2/2 [==============================] - 0s 11ms/step - loss: 15.7023
Epoch 6/500
2/2 [==============================] - 0s 5ms/step - loss: 15.0653
Epoch 7/500
2/2 [==============================] - 0s 14ms/step - loss: 14.4446
Epoch 8/500
2/2 [==============================] - 0s 10ms/step - loss: 13.8368
Epoch 9/500
2/2 [==============================] - 0s 11ms/step - loss: 13.2449
Epoch 10/500
2/2 [==============================] - 0s 5ms/step - loss: 12.6673
Epoch 11/500
2/2 [==============================] - 0s 16ms/step - loss: 12.1056
Epoch 12/500
2/2 [==============================] - 0s 7ms/step - loss: 11.5586
Epoch 13/500
2/2 [=======================

# Backtesting (Irrelevent Now)

In [109]:
# 25th to 31st of December
# this was done while back testing on the 25th to 31st of December week, with training done till 24th of December
X_test_arima = [2230.8902031 , 2190.68793382, 2215.03354658, 2253.06710504,
       2340.83766946, 2322.87380631, 2279.91918307]
X_test_prophet = [2250.278826682667, 2187.6232902138954, 2166.9237947120873, 2216.7738485777973, 2293.2861269535583, 2341.4798982451643, 2282.4565902614195]
X_test = np.array([X_test_arima, X_test_prophet]).T
X_test = np.array(np.log(X_test))
Y_test = [2272.561767578125,
 2231.46533203125,
 2378.739990234375,
 2347.566162109375,
 2300.690673828125,
 2292.0654296875,
 2281.47119140625]


Y_test = np.array(np.log(Y_test))

# Predict 
prediction = model.predict(X_test)
prediction = np.exp(prediction)
Y_test = np.exp(Y_test)
print(prediction)

mae = np.mean(np.abs(prediction - Y_test))
print(mae)
mae2 = np.mean(np.abs(Y_test - X_test_arima))
print(mae2)
mae3 = np.mean(np.abs(Y_test - X_test_prophet))
print(mae3)
# take weighted average of arima and prophet 0.7 and 0.3
avg = 0.2 * np.array(X_test_arima) + 0.8 * np.array(X_test_prophet)
mae4 = np.mean(np.abs(Y_test - avg))
print(mae4)





1/1 [==============================] - 0s 312ms/step
[[2301.4329]
 [2246.3083]
 [2240.4514]
 [2286.3875]
 [2366.4595]
 [2392.719 ]
 [2339.1616]]
58.045175980548365
59.023120571964355
66.6482722934396
63.342087901457816


# Saving the Model


In [114]:
# save model
model.save('model_final.h5')

d:\anaconda3\envs\for_st\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Final Forecasting

In [113]:
model = tf.keras.models.load_model('model_final.h5')
X_a = [2244.71993638, 2393.45040412, 2362.03884959, 2314.69369419,
       2305.66685679, 2295.79723409, 2367.12137716]

X_p = [2268.7283255845828, 2277.9891674705073, 2324.4158400900756, 2324.78965407571, 2311.360588849212, 2288.050187086488, 2343.7584314640208]
X_a = np.array(X_a)
X_p = np.array(X_p)
X_a = np.log(X_a)
X_p = np.log(X_p)


X = 0.75*X_a + 0.25*X_p 
print(X)

X_t = np.array([X_a, X_p]).T

Y = model.predict(X_t)
Y = np.exp(Y)
print(Y)


[7.71899572 7.76813056 7.76326634 7.74812069 7.74374182 7.7379904
 7.76695019]
1/1 [==============================] - 0s 172ms/step
[[2318.3135]
 [2373.174 ]
 [2394.2712]
 [2378.8992]
 [2366.9607]
 [2348.1523]
 [2408.8813]]


Model_first
[2307.4602]
 [2415.6323]
 [2408.0498]
 [2374.5085]
 [2363.9673]
 [2349.7866]
 [2417.6394]

Update model
[[2318.3135]
 [2373.174 ]
 [2394.2712]
 [2378.8992]
 [2366.9607]
 [2348.1523]
 [2408.8813]]